In [1]:
import pandas as pd

kaggle = pd.read_csv('./submissions/submission_1.csv').sort_values('customer_id').reset_index(drop=True)
prepped = pd.read_csv('./submissions/submission_7.csv').sort_values('customer_id').reset_index(drop=True)

kaggle['prediction'] = kaggle['prediction'].apply(lambda s: set(s.split()))
prepped['prediction'] = prepped['prediction'].apply(lambda s: set(s.split()))

df = prepped
df.columns = ['customer_id', 'prepped']
df['kaggle'] = kaggle['prediction']

In [2]:
df['common'] = df.apply(lambda x: x['prepped'] & x['kaggle'], axis=1)
print(df.iloc[0])

customer_id    00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
prepped        {0568601006, 0745232001, 0714790020, 070601600...
kaggle         {0568601006, 0745232001, 0714790020, 070601600...
common         {0568601006, 0745232001, 0714790020, 092424300...
Name: 0, dtype: object


In [3]:
df['uncommon'] = df.apply(lambda x: (x['prepped'] | x['kaggle']) - (x['prepped'] & x['kaggle']), axis=1)

In [4]:
df.loc[0]

customer_id    00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
prepped        {0568601006, 0745232001, 0714790020, 070601600...
kaggle         {0568601006, 0745232001, 0714790020, 070601600...
common         {0568601006, 0745232001, 0714790020, 092424300...
uncommon                                                      {}
Name: 0, dtype: object

In [7]:
user_subgroup_mask = df['uncommon'].apply(lambda x: len(x) > 0)
user_subgroup = df[user_subgroup_mask].reset_index(drop=True)

In [8]:
user_subgroup.head()

,customer_id,prepped,kaggle,common,uncommon
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"{0714790020, 0706016001, 0723529001, 081183500...","{0714790020, 0448509014, 0723529001, 081183500...","{0714790020, 0723529001, 0811835004, 076428000...","{0448509014, 0706016001}"
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"{0852643003, 0351484002, 0794321008, 085264300...","{0852643003, 0714790020, 0794321008, 085264300...","{0852643003, 0794321008, 0852643001, 079432100...","{0714790020, 0351484002}"
2,0000945f66de1a11d9447609b8b41b1bc987ba185a5496...,"{0706016003, 0706016001, 0158340001, 076008400...","{0706016003, 0706016001, 0918522001, 076008400...","{0706016003, 0706016001, 0760084006, 076008400...","{0918522001, 0158340001}"
3,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,"{0914441001, 0807244005, 0706016001, 080724401...","{0706016001, 0918522001, 0807244012, 070601603...","{0706016001, 0807241026, 0807244012, 070601603...","{0914441001, 0807244005, 0918522001, 0778476005}"
4,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,"{0924243002, 0870345002, 0917434002, 090992400...","{0924243002, 0870345002, 0909924004, 091743400...","{0870345002, 0917434002, 0909924004, 072280300...","{0685813003, 0918522001}"


In [10]:
user_subgroup.shape

(397552, 5)

In [11]:
all_uncommon_items = {}
def add_to_set(x):
    for elem in x:
        if elem not in all_uncommon_items:
            all_uncommon_items[elem] = 0
        all_uncommon_items[elem] += 1
user_subgroup['uncommon'].apply(add_to_set)

0         None
1         None
2         None
3         None
4         None
          ... 
397547    None
397548    None
397549    None
397550    None
397551    None
Name: uncommon, Length: 397552, dtype: object

In [12]:
all_uncommon_items

{'0448509014': 17095,
 '0706016001': 106106,
 '0714790020': 11909,
 '0351484002': 506,
 '0918522001': 88224,
 '0158340001': 11898,
 '0914441001': 63,
 '0807244005': 80,
 '0778476005': 2,
 '0685813003': 2304,
 '0924243002': 45353,
 '0923758001': 12110,
 '0669786001': 21,
 '0783346018': 641,
 '0579541001': 22742,
 '0876053003': 65,
 '0516859008': 101,
 '0909370001': 2552,
 '0797710001': 74,
 '0836142001': 176,
 '0715624001': 29310,
 '0706016003': 16869,
 '0739974006': 38,
 '0573085028': 31378,
 '0838644001': 48,
 '0611415005': 557,
 '0907696001': 62,
 '0372860001': 8522,
 '0762846006': 314,
 '0156231001': 2767,
 '0579302001': 80,
 '0825600004': 11,
 '0719655001': 13248,
 '0824337002': 48,
 '0673396002': 3412,
 '0751471001': 29057,
 '0796210001': 776,
 '0803757004': 553,
 '0826508003': 162,
 '0706016002': 2671,
 '0464297007': 1970,
 '0762028005': 5,
 '0936622001': 31,
 '0799365027': 13,
 '0673677002': 33054,
 '0783707047': 120,
 '0658030016': 57,
 '0858052006': 34,
 '0836997011': 9,
 '090

In [13]:
uncommon_df = pd.DataFrame({
    'article_id': list(all_uncommon_items.keys()),
    'count': list(all_uncommon_items.values())
}).sort_values('count', ascending=False).reset_index(drop=True)


In [14]:
uncommon_df.head()

,article_id,count
0,0706016001,106106
1,0918522001,88224
2,0924243002,45353
3,0673677002,33054
4,0573085028,31378


In [15]:
uncommon_df.tail()

,article_id,count
16927,0749674004,1
16928,0808816003,1
16929,0701472005,1
16930,0652924011,1
16931,0873274006,1
